In [152]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import random
import pickle
import requests
import csv

In [3]:
driver = webdriver.Chrome()     # 打开 Chrome 浏览器

# 将刚刚复制的帖在这
driver.get("https://s.1688.com/company/company_search.htm?keywords=%B2%BB%D0%E2%B8%D6%CE%F7%B2%CD%BE%DF&netType=1%2C11&earseDirect=false&button_click=top&n=y&pageSize=30&offset=3&beginPage=1")


In [20]:
with open('all_company_links.pickle', 'rb') as f:
    all_company_links = pickle.load(f)
    
print(len(all_company_links))

2737


In [38]:
# all_company_pages=[]
for link in all_company_links[2593:]:
    res = requests.get(link)
    html = res.text
    all_company_pages.append(html)
    
    #sleep
    if random.random()>0.5:
        time.sleep(0.5+random.random())
    else:
        time.sleep(0.3+random.random())
    
    if random.random()<0.07:
        print(len(all_company_pages))

2599
2601
2605
2623
2629
2631
2657
2712


In [39]:
print(len(all_company_pages))

2737


In [40]:
f=open('all_company_pages.pickle', 'wb')
pickle.dump(all_company_pages, f)
f.close()

In [42]:
with open('all_company_pages.pickle', 'rb') as f:
    company_pages = pickle.load(f)
print(len(company_pages))

2737


In [164]:
mobile_phone_results=[]
name_results=[]
phone_results=[]
address_results=[]
company_name_results=[]
link_results=[]

for idx, company_page in enumerate(company_pages[:]):
    has_mobile_phone=0
    
    mobile_phone = re.findall(r'class="m-mobilephone">[\s\D]*<dt>移动电话：</dt>[\s\D]*<dd>[\s\D]*(\d+)[\s\D]*</dd>', company_page)
    if mobile_phone!=[]:
        if mobile_phone[0]!='0':
            mobile_phone_results.append(mobile_phone[0])
            has_mobile_phone=1
            
    mobile_phone2 = re.findall(r'<dl class="m-mobilephone" data-need-not-login="false" data-no="([\d]+)">', company_page)
    if has_mobile_phone==0 and mobile_phone2!=[]:
        mobile_phone_results.append(mobile_phone2[0])
        has_mobile_phone=1

    if has_mobile_phone:
        name = re.findall(r'人：</dt>[\s\D]*<dd>[\s\D]*.*class="membername" target="_blank">(.*)<', company_page)
        name_results.append(name[0])
        phone = re.findall(r'话：</dt>[\s\D]*<dd>([ \d]*)</dd>', company_page)
        if phone!=[]:
            phone_results.append(phone[0])
        else:
            phone_results.append('无')
            
        address = re.findall(r'class = "address">(.*)[\s\D]*<', company_page)
        address_results.append(address[0])
#         print(address[0])
        company_name = re.findall(r'class="contact-info".*>[\s\D]*<h4>(.*)</h4>', company_page)
        company_name_results.append(company_name[0])
#         print(company_name[0])
        link_results.append(all_company_links[idx])

    

In [165]:
print(len(mobile_phone_results))
print(all_company_links[1])

for x in mobile_phone_results:
    if len(x)!=11:
        print(x)

1823
https://shop7038n7835p7p5.1688.com/page/contactinfo.htm
013702803982


In [166]:
csvfile = open('不锈钢西餐具供应商信息.csv', 'w')
csv_writer = csv.writer(csvfile)
csv_writer.writerow(['手机', '联系人姓名', '公司名称', '公司地址', '电话', '1688网址'])


info_exist=set()
for idx, mobile_phone in enumerate(mobile_phone_results):
    name = name_results[idx]
    if (mobile_phone, name) in info_exist:
        pass
    else:
        info_exist.add((mobile_phone, name))
        company_name = company_name_results[idx]
        address = address_results[idx]
        phone = phone_results[idx]
        link = link_results[idx]
        csv_writer.writerow([mobile_phone, name, company_name, address, phone, link])
        
csvfile.close()